# Dense Passage Retrieval 코드 (in-batch version)

## 1. Install packages and Load data

In [1]:
!pip install datasets
!pip install transformers

/home/minj/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/minj/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/minj/anaconda3/lib/python3.8/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/minj/anaconda3/lib/python3.8/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1") # KorQuAD train 데이터셋을 학습 데이터로 활용합니다.

Reusing dataset squad_kor_v1 (/home/minj/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

## 2. Tokenizer

BERT를 encoder로 사용하므로, huggingface에서 "klue/bert-base" tokenizer를 받아서 사용하자

In [3]:
from transformers import AutoTokenizer
import numpy as np

model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

## 3. Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [5]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [6]:
# Use subset (128 example) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 128)
training_dataset = dataset['train'][sample_idx]

In [7]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, passage를 embedding하는 p_ encoder과 question embedding을 하는 q_encoder 각각에 pre-trained weight 불러오기

In [8]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


In [9]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [10]:
def train(args, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  

  for _ in trange(int(args.num_train_epochs), desc="Epoch"):
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      #print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [11]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01
)

In [12]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Epoch: 100%|██████████| 5/5 [01:09<00:00, 13.84s/it]


## 4. Valid set에 있는 Question에 대하여 passage retrieval를 해보자!

valid set을 불러오자

In [21]:
from pprint import pprint
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10] # 10개 documents중에서 top k를 뽑아보자
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(f"*********[Selected Query]*********")
pprint(query)
print()
print(f"*********[Ground Truth]*********")
pprint(ground_truth)

*********[Selected Query]*********
'지상파 3사중 유일하게 SBS만 단독중계를 하게 된 이유는 무엇때문인가?'

*********[Ground Truth]*********
('한편 대한민국 내 방송사로는 유일하게 SBS에서 단독으로 밴쿠버 올림픽 개·폐회식 및 경기 등을 생중계하였다. SBS를 제외한 나머지 '
 '지상파 방송사들은 아예 밴쿠버 현지에 취재진 등을 파견하거나 올림픽에 관한 방송을 하지 않기로 방침을 정했으나, 시청자들의 항의로 '
 'SBS에서 제공한 2분분량의 영상으로 관련보도를 하는데 합의하였다. SBS는 IOC와의 올림픽 중계 독점계약 절차에 따라 향후 2012년 '
 '영국 런던과 2016년 브라질 리우데자네이루에서 개최되는 하계 올림픽과 2014년 러시아 소치에서 열리는 동계 올림픽에서도 올림픽 '
 '단독중계를 하게 될 예정이었다. 이에 KBS와 MBC 등 방송사들이 방송통신위원회에 공동 중계요청을 제안했으나 SBS측이 IOC와의 '
 '독점계약상에 위반된다는 이유로 이를 거부하여 결국 SBS 단독으로 올림픽 중계를 하게 되었다. 그러나 방송통신위원회의 징계로 SBS가 '
 '과징금을 물게 되어 2012년 영국 런던과 2016년 브라질 리우데자네이루에서 개최되는 하계 올림픽과 2014년 러시아 소치에서 열리는 '
 '동계 올림픽 부터 독점중계를 깨고 지상파 3사가 공동 중계하게 되었다. 한편 SBS는 올림픽 생중계 도중 스피드 스케이팅 중계 때 '
 '대한민국의 박도영 선수를 소개하는 자막에서 태극기와 대한민국(KOR)이 아닌 일본의 일장기와 일본(JPN) 국적으로 오표기를 하여 논란을 '
 '일으키기도 했다. 이에 SBS 게시판에는 항의의 글들이 쇄도했으며 SBS는 이에 대해 공식적으로 사과와 해명을 하였다.')


앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [22]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [23]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


dot product를 수행 => Document들의 유사도 ranking을 구함

In [24]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 11])
tensor([[177.5575, 179.3276, 170.4498, 164.7650, 171.7243, 182.1234, 173.9379,
         159.2195, 170.6320, 185.8634, 200.5819]])
tensor([10,  9,  5,  1,  0,  6,  4,  8,  2,  3,  7])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [25]:
k = 5
print("*********[Search query]*********\n", query, "\n")
print("*********[Ground truth passage]*********")
print(ground_truth, "\n")
print('************************************')
for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])
  print()

*********[Search query]*********
 지상파 3사중 유일하게 SBS만 단독중계를 하게 된 이유는 무엇때문인가? 

*********[Ground truth passage]*********
한편 대한민국 내 방송사로는 유일하게 SBS에서 단독으로 밴쿠버 올림픽 개·폐회식 및 경기 등을 생중계하였다. SBS를 제외한 나머지 지상파 방송사들은 아예 밴쿠버 현지에 취재진 등을 파견하거나 올림픽에 관한 방송을 하지 않기로 방침을 정했으나, 시청자들의 항의로 SBS에서 제공한 2분분량의 영상으로 관련보도를 하는데 합의하였다. SBS는 IOC와의 올림픽 중계 독점계약 절차에 따라 향후 2012년 영국 런던과 2016년 브라질 리우데자네이루에서 개최되는 하계 올림픽과 2014년 러시아 소치에서 열리는 동계 올림픽에서도 올림픽 단독중계를 하게 될 예정이었다. 이에 KBS와 MBC 등 방송사들이 방송통신위원회에 공동 중계요청을 제안했으나 SBS측이 IOC와의 독점계약상에 위반된다는 이유로 이를 거부하여 결국 SBS 단독으로 올림픽 중계를 하게 되었다. 그러나 방송통신위원회의 징계로 SBS가 과징금을 물게 되어 2012년 영국 런던과 2016년 브라질 리우데자네이루에서 개최되는 하계 올림픽과 2014년 러시아 소치에서 열리는 동계 올림픽 부터 독점중계를 깨고 지상파 3사가 공동 중계하게 되었다. 한편 SBS는 올림픽 생중계 도중 스피드 스케이팅 중계 때 대한민국의 박도영 선수를 소개하는 자막에서 태극기와 대한민국(KOR)이 아닌 일본의 일장기와 일본(JPN) 국적으로 오표기를 하여 논란을 일으키기도 했다. 이에 SBS 게시판에는 항의의 글들이 쇄도했으며 SBS는 이에 대해 공식적으로 사과와 해명을 하였다. 

************************************
Top-1 passage with score 200.5819
한편 대한민국 내 방송사로는 유일하게 SBS에서 단독으로 밴쿠버 올림픽 개·폐회식 및 경기 등을 생중계하였다. SBS를 제외